# Extracting the training data

In [1]:
import pandas as pd
import mediapipe as mp
import numpy as np
import cv2
from pathlib import Path

In [2]:
# Autoreload when script is changed
%load_ext autoreload
%autoreload 2

## Single image
All steps to take on each image.

In [4]:
from preprocessing import load_image,  extract_features

In [5]:
results=load_image("DATA/training data/training frames/Adho mukha svanasana - Downward-facing Dog/out1891.png")

In [116]:
def get_landmark(
    name: str, landmarks: mp.framework.formats.landmark_pb2.NormalizedLandmarkList
):
    """Get landmark postition from English name."""
    landmark_names = [
        "nose",
        "left_eye_inner",
        "left_eye",
        "left_eye_outer",
        "right_eye_inner",
        "right_eye",
        "right_eye_outer",
        "left_ear",
        "right_ear",
        "mouth_left",
        "mouth_right",
        "left_shoulder",
        "right_shoulder",
        "left_elbow",
        "right_elbow",
        "left_wrist",
        "right_wrist",
        "left_pinky_1",
        "right_pinky_1",
        "left_index_1",
        "right_index_1",
        "left_thumb_2",
        "right_thumb_2",
        "left_hip",
        "right_hip",
        "left_knee",
        "right_knee",
        "left_ankle",
        "right_ankle",
        "left_heel",
        "right_heel",
        "left_foot_index",
        "right_foot_index",
    ]
    landmark_i = landmarks.landmark[landmark_names.index(name)]
    return np.array([landmark_i.x, landmark_i.y, landmark_i.z])

In [117]:
get_landmark("right_heel",results)

array([0.21777244, 0.82397199, 0.02456942])

In [118]:
landmarks=results

Scale by torso size

In [119]:
shoulder_center = (
        get_landmark("left_shoulder", landmarks)
        - get_landmark("right_shoulder", landmarks)
    ) / 2.
hip_center = (
    get_landmark("left_hip", landmarks) - get_landmark("right_hip", landmarks)
) / 2.
torso_length=np.linalg.norm(shoulder_center - hip_center)
torso_length

0.03380091851357698

Position

In [120]:
hip_center=np.mean()

TypeError: _mean_dispatcher() missing 1 required positional argument: 'a'

In [8]:
extract_features(results)

{'Nose_CenterShoulders_CenterHips': 142.967718796389,
 'LeftHip_LeftShoulder_LeftElbow': 119.11713203986034,
 'RightHip_RightShoulder_RightElbow': 164.50457492847735,
 'LeftShoulder_LeftElbow_LeftWrist': 96.51088868646956,
 'RightShoulder_RightElbow_RightWrist': 142.5128476540785,
 'LeftShoulder_CenterShoulders_Nose': 100.78660874705022,
 'RightShoulder_CenterShoulders_Nose': 79.21339125294979,
 'LeftIndex1_LeftWrist_LeftElbow': 163.5508086741753,
 'RightIndex1_RightWrist_RightElbow': 127.2773914593761,
 'LeftShoulder_LeftHip_LeftKnee': 70.30118526951661,
 'RightShoulder_RightHip_RightKnee': 78.76055831085836,
 'LeftHip_LeftKnee_LeftAnkle': 162.33005532600495,
 'RightHip_RightKnee_RightAnkle': 170.65806093330588,
 'LeftKnee_LeftAnkle_LeftFootIndex': 89.57104207809206,
 'RightKnee_RightAnkle_RightFootIndex': 90.42606929039493,
 'LeftElbow_CenterShoulders_RightElbow': 125.485441111933,
 'LeftKnee_CenterHips_RightKnee': 52.268853969661315}

## Folder

In [9]:
data_dir=Path("DATA/training data/training frames/Ananda Balasana - Happy Baby's pose/")

In [10]:
data=[]
for img_file in data_dir.rglob("*.png"):
    result = {}
    try:
#         split by "-", remerge english splits
        names = img_file.parent.name.split("-")
        if len(names)>1:
            name_sa=names[0].strip()
            name_en='-'.join(names[1:]).strip()
        else:
            name_sa=""
            name_en=img_file.parent.name.strip()
    except:
        continue
        
    result["path"]=img_file.relative_to(data_dir)
    result["name_en"] = name_en
    result["name_sa"] = name_sa
    features = extract_features(normalize_body(load_image(str(img_file.resolve()))))
    result.update(features)
    data.append(result)
df=pd.DataFrame(data)

df

NameError: name 'normalize_body' is not defined

In [58]:
df.value_counts("name_en")

name_en
Easy Pose                125
Low Lunge                114
Half Splits Pose          98
Downward-facing Dog       46
Standing Forward Bend     31
Andrea's Cat              27
.ipynb_checkpoints        24
Child's pose              23
high plank                21
Happy Baby's pose         19
Upward-Facing Dog         18
Cobra Pose                 9
cow pose                   5
cat pose                   4
dtype: int64

## Check preprocessed data

In [133]:
ds=pd.read_csv("output.csv")
ds

,Unnamed: 0,name_en,name_sa,nose_x,nose_y,nose_z,left_shoulder_x,left_shoulder_y,left_shoulder_z,right_shoulder_x,...,left_ankle_z,right_ankle_x,right_ankle_y,right_ankle_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z
0,0,Half Splits Pose,Ardha Hanumanasana,0.581890,0.527894,-0.168950,0.544566,0.609070,0.089030,0.514745,...,0.100718,0.579826,0.870550,-0.076587,0.261653,0.832126,0.029716,0.618466,0.829347,-0.148282
1,1,Half Splits Pose,Ardha Hanumanasana,0.562555,0.646910,-0.019731,0.513664,0.610593,0.159394,0.517667,...,0.193861,0.570697,0.859199,0.046980,0.267511,0.828272,0.152652,0.616064,0.836311,0.023429
2,2,Half Splits Pose,Ardha Hanumanasana,0.579493,0.534021,-0.155556,0.545002,0.612555,0.089005,0.516335,...,0.092996,0.580156,0.868410,-0.066033,0.262553,0.832726,0.021859,0.620628,0.828138,-0.135098
3,3,Half Splits Pose,Ardha Hanumanasana,0.575249,0.512026,0.035455,0.518560,0.477142,-0.051971,0.545835,...,0.274690,0.272477,0.812547,0.060813,0.599598,0.818101,0.311058,0.241514,0.855104,0.084337
4,4,Half Splits Pose,Ardha Hanumanasana,0.575239,0.504725,-0.107558,0.542487,0.616061,0.086450,0.516042,...,0.078715,0.578084,0.864819,-0.083012,0.262293,0.832367,-0.005233,0.617623,0.830452,-0.166826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,535,high plank,Phalakasana,0.657534,0.647592,-0.064799,0.601228,0.600332,0.140197,0.602093,...,0.343990,0.275735,0.781733,-0.053675,0.307514,0.826304,0.307695,0.290173,0.850512,-0.141375
536,536,high plank,Phalakasana,0.629904,0.672581,-0.131697,0.575665,0.615809,0.077150,0.573396,...,0.372269,0.259888,0.802552,-0.033931,0.302700,0.831161,0.327971,0.286621,0.862103,-0.125585
537,537,high plank,Phalakasana,0.636387,0.667703,-0.091850,0.582730,0.609366,0.117416,0.577078,...,0.413961,0.255379,0.802033,-0.104536,0.285115,0.824037,0.392816,0.252718,0.865890,-0.193863
538,538,high plank,Phalakasana,0.642756,0.673534,-0.116394,0.595392,0.609434,0.103701,0.599443,...,0.262926,0.277865,0.787411,-0.055811,0.302497,0.823669,0.183150,0.285557,0.857354,-0.164851


In [135]:
ds.isna().sum()

Unnamed: 0             0
name_en                0
name_sa               27
nose_x                 3
nose_y                 3
nose_z                 3
left_shoulder_x        3
left_shoulder_y        3
left_shoulder_z        3
right_shoulder_x       3
right_shoulder_y       3
right_shoulder_z       3
left_elbow_x           3
left_elbow_y           3
left_elbow_z           3
right_elbow_x          3
right_elbow_y          3
right_elbow_z          3
left_wrist_x           3
left_wrist_y           3
left_wrist_z           3
right_wrist_x          3
right_wrist_y          3
right_wrist_z          3
left_index_1_x         3
left_index_1_y         3
left_index_1_z         3
right_index_1_x        3
right_index_1_y        3
right_index_1_z        3
left_thumb_2_x         3
left_thumb_2_y         3
left_thumb_2_z         3
right_thumb_2_x        3
right_thumb_2_y        3
right_thumb_2_z        3
left_hip_x             3
left_hip_y             3
left_hip_z             3
right_hip_x            3


In [136]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 60 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          540 non-null    int64  
 1   name_en             540 non-null    object 
 2   name_sa             513 non-null    object 
 3   nose_x              537 non-null    float64
 4   nose_y              537 non-null    float64
 5   nose_z              537 non-null    float64
 6   left_shoulder_x     537 non-null    float64
 7   left_shoulder_y     537 non-null    float64
 8   left_shoulder_z     537 non-null    float64
 9   right_shoulder_x    537 non-null    float64
 10  right_shoulder_y    537 non-null    float64
 11  right_shoulder_z    537 non-null    float64
 12  left_elbow_x        537 non-null    float64
 13  left_elbow_y        537 non-null    float64
 14  left_elbow_z        537 non-null    float64
 15  right_elbow_x       537 non-null    float64
 16  right_el

In [139]:
ds.value_counts("name_en")

name_en
Easy Pose                125
Low Lunge                114
Half Splits Pose          98
Downward-facing Dog       46
Standing Forward Bend     31
Andrea's Cat              27
Child's pose              23
high plank                21
Happy Baby's pose         19
Upward-Facing Dog         18
Cobra Pose                 9
cow pose                   5
cat pose                   4
dtype: int64